In [1]:
%load_ext autoreload
%autoreload 2
from utils import *
from IPython.display import display, HTML

# Variants

In [8]:
nature13908_vars = "/Users/jiayao/Work/BrainDisorders/data/nature13908-s2/Supplementary_Table_2.xlsx"
all_var = pd.read_excel(nature13908_vars)
display(all_var.head(2))
LGD = set(["noStart", "frame-shift", "nonsense", "noEnd", "no-frame-shift-newStop","splice-site"])
MIS = set(["missense"])
SYN = set(["synonymous"])
InPro = set(["pMsM", "pMsF", "pFsF", "pF", "pM", "pFsM"])
InSib = set(["pMsM", "pMsF", "pFsF", "sF", "sM", "pFsM"])

,familyId,location,variant,vcfVariant,inChild,fromParent,effectGene,effectType,familyDescription,CSHL,YALE,UW,IossifovWE2012,EichlerWE2012,StateWE2012,EichlerTG2012
0,14505,1:897349,sub(G->A),1:897349:G:A,pM,NaN,KLHL17,synonymous,pMsM,NaN,NaN,weak:valid,NaN,NaN,NaN,NaN
1,14320,3:127395222,sub(T->G),3:127395222:T:G,pF,NaN,ABTB1,noStart,pFsM,strong,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
DatDir = "/Users/jiayao/Work/BrainDisorders/data/DenovoVariants/"
ProLGD = all_var[(all_var["inChild"].isin(InPro)) & (all_var["effectType"].isin(LGD))]
ProMIS = all_var[(all_var["inChild"].isin(InPro)) & (all_var["effectType"].isin(MIS))]
ProSYN = all_var[(all_var["inChild"].isin(InPro)) & (all_var["effectType"].isin(SYN))]
SibLGD = all_var[(all_var["inChild"].isin(InSib)) & (all_var["effectType"].isin(LGD))]
SibMIS = all_var[(all_var["inChild"].isin(InSib)) & (all_var["effectType"].isin(MIS))]
SibSYN = all_var[(all_var["inChild"].isin(InSib)) & (all_var["effectType"].isin(SYN))]
SibNSYN = all_var[(all_var["inChild"].isin(InSib)) & (all_var["effectType"].isin(MIS.union(LGD)))]
ProLGD.to_excel(DatDir + "wigler2014ProLGD.xlsx", index=False)
ProMIS.to_excel(DatDir + "wigler2014ProMIS.xlsx", index=False)
ProSYN.to_excel(DatDir + "wigler2014ProSYN.xlsx", index=False)
SibLGD.to_excel(DatDir + "wigler2014SibLGD.xlsx", index=False)
SibMIS.to_excel(DatDir + "wigler2014SibMIS.xlsx", index=False)
SibSYN.to_excel(DatDir + "wigler2014SibSYN.xlsx", index=False)
SibNSYN.to_excel(DatDir + "wigler2014SibNSYN.xlsx", index=False)

In [24]:
x = "pMsM"
x[-2:]

'sM'

In [25]:
GeneCount = ProLGD.groupby("effectGene")["effectGene"].count()
ProLGD.insert(list(ProLGD.columns.values).index("effectGene")+1, "GeneCount", ProLGD.apply
                 (lambda row: GeneCount[row["effectGene"]], axis=1), allow_duplicates=False )
ProLGD.head()
RecProLGD = ProLGD[ProLGD["GeneCount"]>=2]

In [26]:
RecProLGD.head()

,familyId,location,variant,vcfVariant,inChild,fromParent,effectGene,GeneCount,effectType,familyDescription,CSHL,YALE,UW,IossifovWE2012,EichlerWE2012,StateWE2012,EichlerTG2012
108,13545,20:49509095,ins(T),20:49509094:G:GT,pM,NaN,ADNP,2,frame-shift,pM,NaN,strong:valid,NaN,NaN,NaN,NaN,yes
123,12130,20:49510028,del(2),20:49510027:CTT:C,pF,NaN,ADNP,2,frame-shift,pF,NaN,NaN,strong:valid,NaN,yes,NaN,NaN
242,12645,4:114232545,sub(C->T),4:114232545:C:T,pM,NaN,ANK2,3,nonsense,pMsM,strong:valid,NaN,NaN,yes,NaN,NaN,NaN
259,14256,4:114251469,sub(C->T),4:114251469:C:T,pM,NaN,ANK2,3,nonsense,pMsF,NaN,strong:valid,NaN,NaN,NaN,NaN,NaN
264,13768,4:114277600,del(1),4:114277599:AG:A,pM,NaN,ANK2,3,frame-shift,pMsF,strong,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
print RecProLGD.shape

(67, 17)


In [28]:
RecProLGD.to_excel(DatDir + "wigler2014RecProLGD.xlsx", index=False)

# Go Term

In [43]:
# Get http://geneontology.org/ontology/go-basic.obo
from goatools.base import download_go_basic_obo
obo_fname = download_go_basic_obo()

# Get ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2go.gz
from goatools.base import download_ncbi_associations
gene2go = download_ncbi_associations()

from goatools.associations import read_ncbi_gene2go

go2geneids_human = read_ncbi_gene2go("gene2go", taxids=[9606], go2geneids=True)
print("{N} GO terms associated with human NCBI Entrez GeneIDs".format(N=len(go2geneids_human)))

from goatools.go_search import GoSearch

srchhelp = GoSearch("go-basic.obo", go2items=go2geneids_human)

  wget.download(http://purl.obolibrary.org/obo/go/go-basic.obo out=go-basic.obo)
  wget.download(ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2go.gz out=gene2go.gz)


  gunzip gene2go.gz
  17,983 items READ: gene2go
17983 GO terms associated with human NCBI Entrez GeneIDs
go-basic.obo: fmt(1.2) rel(2018-10-23) 47,358 GO Terms; optional_attrs(comment def relationship synonym xref)


In [30]:
from goatools import obo_parser
import wget
import os
go_obo_url = 'http://purl.obolibrary.org/obo/go/go-basic.obo'
data_folder = os.getcwd() + '/data'

# Check if we have the ./data directory already
if(not os.path.isfile(data_folder)):
    # Emulate mkdir -p (no error if folder exists)
    try:
        os.mkdir(data_folder)
    except OSError as e:
        if(e.errno != 17):
            raise e
else:
    raise Exception('Data path (' + data_folder + ') exists as a file. '
                   'Please rename, remove or change the desired location of the data path.')

def find_children(term1, go, go_term_set={}, ret=False):
    for term2 in term1.children:
        go_term_set.update({term2})
        
        # Recurse on term to find all children
        find_children(term2, go, go_term_set)
    if(ret):
        return go_term_set
    
# Check if the file exists already
if(not os.path.isfile(data_folder+'/go-basic.obo')):
    go_obo = wget.download(go_obo_url, data_folder+'/go-basic.obo')
else:
    go_obo = data_folder+'/go-basic.obo'
go = obo_parser.GODag(go_obo)

/Users/jiayao/Work/BrainDisorders/src/data/go-basic.obo: fmt(1.2) rel(2018-10-19) 47,359 GO Terms


In [100]:
def GetALLGo(GoID):
    Root = go[GoID]
    all_go = Root.get_all_children()
    all_go.add(GoID)
    return all_go

In [101]:
chromatin_organization = "GO:0006325"
synaptic_signaling = "GO:0099536"
synapse = "GO:0045202"
cortical_cytoskeleton = "GO:0030863"
presynaptic_cytoskeleton = "GO:0099569"
postsynaptic_cytoskeleton = "GO:0099571"
channel_activity = "GO:0015267"

In [102]:
goset_chromatin_organization = GetALLGo(chromatin_organization)
goset_synaptic_signaling = GetALLGo(synaptic_signaling)
goset_synapse = GetALLGo(synapse)
goset_presynaptic_cytoskeleton = GetALLGo(presynaptic_cytoskeleton)
goset_postsynaptic_cytoskeleton = GetALLGo(postsynaptic_cytoskeleton)
goset_cortical_cytoskeleton = GetALLGo(cortical_cytoskeleton)
goset_channel_activity = GetALLGo(channel_activity)

In [103]:
chromatin_geneids = srchhelp.get_items(goset_chromatin_organization)
synaptic_signaling_geneids = srchhelp.get_items(goset_synaptic_signaling)
synapse_geneids = srchhelp.get_items(goset_synapse)
presynaptic_cyto_geneids = srchhelp.get_items(goset_presynaptic_cytoskeleton)
postsynaptic_cyto_geneids = srchhelp.get_items(goset_postsynaptic_cytoskeleton)
cortical_cytoskeleton_geneids = srchhelp.get_items(goset_cortical_cytoskeleton)
channel_geneids = srchhelp.get_items(goset_channel_activity)

In [104]:
print len(chromatin_geneids)
print len(synaptic_signaling_geneids)
print len(synapse_geneids)
print len(presynaptic_cyto_geneids)
print len(postsynaptic_cyto_geneids)
print len(cortical_cytoskeleton_geneids)
print len(channel_geneids)

713
324
714
8
7
74
467


In [105]:
cyto_sig = presynaptic_cyto_geneids.union(postsynaptic_cyto_geneids).union(cortical_cytoskeleton_geneids).union(synaptic_signaling_geneids)
chromatin = chromatin_geneids
synapse = synapse_geneids
channel = channel_geneids

In [106]:
dat_dir = '/Users/jiayao/Work/BrainDisorders/data/functions/'
def to_file(filename, _list):
    fout = open(filename, 'wb')
    for term in _list:
        fout.write("{}\n".format(term))
#print chromatin_geneids

to_file(dat_dir+"go_cyto_sig.list", cyto_sig)
to_file(dat_dir+"go_chromatin.list", chromatin)
to_file(dat_dir+"go_synapse.list", synapse)
to_file(dat_dir+"go_channel.list", channel)